In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests

# Import the API key.
from config import gkey as g_key

In [2]:
# Review for the first cell (above ^^^)...
# pandas dependency to read CSV file, and create the locations and measurements
#  from the DataFrame.
# gmaps and the API key to create heatmaps and the locations map
# requests to make a request to the Google Places JSON file; this will allow us
#  to get hotel locations from the latitude and longitude of the city.

# next, we'll read the `cities.csv` file into a DataFrame...

In [3]:
# Store the CSV file ...saved (created) in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Port Alfred,ZA,2022-07-08 00:13:28,-33.5906,26.8910,60.62,70,21,6.64
1,1,Ciudad Bolivar,VE,2022-07-08 00:13:28,8.1222,-63.5497,80.19,69,77,9.91
2,2,Carnarvon,AU,2022-07-08 00:13:29,-24.8667,113.6333,63.82,26,0,8.93
3,3,Mataura,NZ,2022-07-08 00:13:29,-46.1927,168.8643,47.48,92,100,14.52
4,4,Port Victoria,KE,2022-07-08 00:13:30,0.0939,33.9756,69.17,70,62,8.86


In [4]:
# NOTE: a "caveat to using gmaps: The data we use for any mapping must be either
#       an integer or a floating-point decimal number. Let's check the data types
#       for the columns of our DataFrame.
#
# REWIND: Recall that ...use the `dtypes` method to get the data types of a
#         DataFrame.
# Confirm (that) the data types for the data columns are integers or floating-
# point decimal numbers."

# Get the data types.
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [5]:
city_data_df[(city_data_df["Country"].isnull()==True)]

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
224,224,Aranos,NaN,2022-07-08 00:18:33,-24.1333,19.1167,55.44,29,0,12.44
322,322,Luderitz,NaN,2022-07-08 00:22:00,-26.6481,15.1594,59.58,48,0,5.57
398,398,Walvis Bay,NaN,2022-07-08 00:23:29,-22.9575,14.5053,54.55,62,0,5.75


In [6]:
# 6.5.2 Create Heatmaps for Weather Parameters

# --Create a Maximum Temperature Heatmap
# "First, tell gmaps to use your API key. ..only need to configure gmaps to
# use your API key once."
#
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

# "Next, create the heatmap for the maximum temperature. The general syntax for
# creating a heatmap is as follows." (cell below)...

In [7]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]

# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]

# Assign the figure variable.
fig = gmaps.figure()

# Assign the heatmap variable.
#heat_layer = gmaps.heatmap_layer(locations, weights=max_temp) #as at first run
# Assign the heatmap variable; as revised from line 'commented out' above ^^^...
##heat_layer = gmaps.heatmap_layer(locations, weights=temps)
# Assign again, using list comprehension rather than (so 'comment out' both)
# cell above and code line above... code line below had ### comment, but now
# keeping it as sole uncommented `heat_layer` assignment code
heat_layer = gmaps.heatmap_layer(locations,
                    weights=[max(temp,0) for temp in max_temp])

# Add the heatmap layer.
fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# running cell above results in above ^^^ "large landscape map in the output
# window."

# "Google heatmaps do not plot negative numbers. If you have a maximum
# temperature that is less than 0 °F, then you will get an 
# `InvalidWeightException` error for this line of code: 
#     `heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)`
#
# To remove the negative temperatures we can use a `for` loop to iterate
# through the `max_temp` and add the temperatures that are greater than
# 0 °F to a new list.
#
# Add a new cell above our previous code block, and then add the code as is
# into the cell and run the cell." ... ^^^ ^^^

In [9]:
# nb for "# Heatmap of temperature" cell initially received "This page can't
# load Google Maps correctly." .."Do you own this website? [OK]" message
#
# "The common errors for this ^^^ are:
#  - Not configuring gmaps to use your API key with
#      `gmaps.configure(api_key=g_key)`
#  - Not importing your API key from the correct folder
#  - Not having or using the correct API key for gmaps:
#  - Not adding a credit card to your account for billing
#
# Other options can be found by clicking on 'Do you own this website?' in the
# pop-up message window. ..For more information, see the _documentation on
# error messages_ [web url: https://developers.google.com/maps/documentation/
#                                   javascript/error-messages?
#                                   utm_source=maps_js&utm_medium=degraded&
#                                   utm_campaign=keyless#
#                                   api-key-and-billing-errors]

In [10]:
# The map (above) is too large (has to repeat in order to fill area). We
# will need to make some adjustments to the `gmaps.figure()` attribute.

# --Adjust Heatmap Zoom, Intensity, and Point Radius
#
# First, add the geographic center of Earth in the form of latitude and
# longitude (30.0° N and 31.0° E). Also, add a zoom level so that only
# one map of Earth is shown. ..When we add a center and zoom level to
# the `gmaps.figure()` attribute, it will look like this:
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

#fig.add_layer(heat_layer)
fig
# "You might have to play with the zoom level to get things right.
# ..When we run the cell, our map is centered and shows all the
# inhabitable places on Earth."

Figure(layout=FigureLayout(height='420px'))

In [11]:
# "If we scroll and zoom in on North America, we can see that the circles
# for the temperatures need to be modified so that they are larger and
# show temperature gradient differences.
#
# If you review the gmaps documentation, you may notice there is a
# dissipation option for creating heatmaps that can be added to the
# `gmaps.heat_layer()` attribute.
#
# The options for this are:
#  1. The default option for the dissipation is 'True,' so we need
#      to set our 'dissipation' to 'False.'
#  2. We can add `max_intensity` to make each measurement have a
#      better gradient variance.
#  3. We can add `point_radius` to make each measurement radius
#      larger.
#
# Also, you'll have to tweak these values until you are satisfied
# with the final map. ..Let's modify our `gmaps.heat_layer()`
# attribute to `heat_layer = gmaps.heatmap_layer(locations,
#                 weights=[max(temp,0) for temp in max_temp],
#                 dissipating=False, max_intensity=300,
#                 point_radius=4)`
# and rerun our code."


# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]

# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]

# Assign the figure variable.
####fig = gmaps.figure()

# Assign the heatmap variable.
#heat_layer = gmaps.heatmap_layer(locations, weights=max_temp) #as at first run
# Assign the heatmap variable; as revised from line 'commented out' above ^^^...
##heat_layer = gmaps.heatmap_layer(locations, weights=temps)
# Assign again, using list comprehension rather than (so 'comment out' both)
# cell above and code line above...
heat_layer = gmaps.heatmap_layer(locations,
                    weights=[max(temp,0) for temp in max_temp],
                    dissipating=False, max_intensity=300, point_radius=4)

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Add the heatmap layer.
fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
# As for map figure above ^^^ ...
# "If we scroll and zoom in on North America, we can see that the circles for
# the temperature are larger and the gradients show differences. ..Now our
# maximum temperature heat map looks a lot better!"

# NOTE: "If you see the following message in the output window below your
#        code block, when you open you Jupyter Notebook file that had the
#        Google map, you will have (to) rerun the cell to create the map."

#A Jupyter widget could not be displayed because the widget state could not
#be found. This could happen if the kernel storing the widget is no longer
#available, or if the widget state was not saved in the notebook. You may
#be able to create the widget by running the appropriate cells."

In [13]:
# --Create a Percent Humidity Heatmap
#
# Now that we have created our maximum temperature heatmap, let's create the
# heatmap for humidity. We can reuse the code and use the humidity values for
# the measurements." ...

# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=False, max_intensity=300,
                                point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
# --Create a Percent Cloudiness Heatmap
#
# "Next, we will create the heatmap for percent cloudiness.

# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig=gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds,
                                dissipating=False, max_intensity=300,
                                point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
# --Create a Wind Speed Heatmap
#
# "Now we can create the final heatmap. copy the code that created the
# previous heatmap and edit your code with the line
# `wind = city_data_df["Wind Speed"]`
# so that the wind speed replaces the percent cloudiiness, and set the
# variable `weights=wind`, and run the cell." ... "Congratulations on
# creating the heatmaps for each weather parameter!"

# Heatmap of Wind Speed
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind,
                                dissipating=False, max_intensity=300,
                                point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
# 6.5.3 Get Vacation Criteria

# Create app: prompt viewer to enter minimum and maximum temperature
# ranges (will convert to floating-point decimal numbers) to filter
# the `city_data_df` DataFrame.

# Prompt customer to add a minimum and maximum temperature value.
try:
    min_temp = float(input(
      "What is the minimum temperature you would like for your trip? "))
except:
    min_temp = 75.0
    print(f"... will use default min of {min_temp} °F")

try: 
    max_temp = float(input(
      "What is the maximum temperature you would like for your trip? "))
except:
    max_temp = 90.0
    print(f"... will use default max of {max_temp} °F")

What is the minimum temperature you would like for your trip? 
... will use default min of 75.0 °F
What is the maximum temperature you would like for your trip? 
... will use default max of 90.0 °F


In [17]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[
                        (city_data_df["Max Temp"] <= max_temp) &
                        (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Ciudad Bolivar,VE,2022-07-08 00:13:28,8.1222,-63.5497,80.19,69,77,9.91
18,18,College,US,2022-07-08 00:13:34,64.8569,-147.8028,79.09,43,75,0.00
25,25,Butaritari,KI,2022-07-08 00:13:35,3.0707,172.7902,81.95,77,86,15.52
28,28,Isangel,VU,2022-07-08 00:13:36,-19.5500,169.2667,81.07,72,81,5.99
37,37,Avarua,CK,2022-07-08 00:13:40,-21.2078,-159.7750,77.05,61,20,8.05
38,38,Gurupi,BR,2022-07-08 00:09:23,-11.7292,-49.0686,75.74,40,0,3.56
39,39,Ixtapa,MX,2022-07-08 00:13:40,20.7000,-105.2000,86.23,79,75,9.22
42,42,Atuona,PF,2022-07-08 00:10:20,-9.8000,-139.0333,77.63,74,3,22.01
44,44,Meulaboh,ID,2022-07-08 00:14:18,4.1363,96.1285,75.42,90,52,4.45
46,46,Tautira,PF,2022-07-08 00:14:42,-17.7333,-149.1500,78.51,73,3,10.42


In [18]:
# To determine if any null values for the rows in the
# `preferred_cities_df` DataFrame can choose from the
# following options ...
#   preferred_cities_df.count()         #check all have same #?
#   preferred_cities_df.isnull().sum()  #check for non-zeros
#   preferred_cities_df.notnull().sum() #check all have same #?

preferred_cities_df.count()

City_ID       212
City          212
Country       212
Date          212
Lat           212
Lng           212
Max Temp      212
Humidity      212
Cloudiness    212
Wind Speed    212
dtype: int64

In [19]:
# For cities count ^^^, note "the following guidance:
#  1. Depending on the time of year and the seasons,
#     ...might have to adjust the minimum and maximum
#     temperature to get enough cities.
#  2. It is a good idea to keep the number of cities
#     to fewer than 200 to make it easier to plot the
#     markers on the heatmap.
#  3. If ... some rows with null values, ... need to
#     drop them using the `dropna()` method at the
#     end of ...(the) filtering statement when ...
#     creating the new DataFrame."

# "Now that "we have all the cities the customer
# wants to travel to, (next) ...will need to find a
# hotel to stay in the city." ...

In [20]:
# 6.5.4 Map Vacation Criteria

# have now filter of database (DataFrame) based on
# temperature preferences, next show heatmap for
# maximum temperature for the filtered cities; and
# also create marker for each city that will display
# "name of the city, country code, maximum temperature,
# and name of a nearby hotel within three miles of the
# coordinates when the marker is clicked."

# "Using the coordinates from the `preferred_cities_df`
# DataFrame, find a hotel using our Google Places API
# and then retrieve that hotel information. Once we
# retrieve the hotel information, we'll need to store
# it so we can reference it and add the information to
# the pop-up marker."

# --Get Travel Destinations
#
# won't add the hotel info to `preferred_cities_df` b/c
# this DataFrame is filtered ("customer will always
# filter it for each trip"). "We'll need to create a new
# DataFrame specifically for the data needed to create a
# heatmap and pop-up markers."

# Create DataFrame called hotel_df to store hotel names along
# with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp",
                                "Lat", "Lng"]].copy()
# next "Add a new column to the `hotel_df` DataFrame to hold
#       the name of the hotel."
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Ciudad Bolivar,VE,80.19,8.1222,-63.5497,
18,College,US,79.09,64.8569,-147.8028,
25,Butaritari,KI,81.95,3.0707,172.7902,
28,Isangel,VU,81.07,-19.5500,169.2667,
37,Avarua,CK,77.05,-21.2078,-159.7750,
38,Gurupi,BR,75.74,-11.7292,-49.0686,
39,Ixtapa,MX,86.23,20.7000,-105.2000,
42,Atuona,PF,77.63,-9.8000,-139.0333,
44,Meulaboh,ID,75.42,4.1363,96.1285,
46,Tautira,PF,78.51,-17.7333,-149.1500,


In [21]:
# Next, will use, with "the latitude and longitude and
# specific parameters, .. the Google Places Nearby Search
# request to retrieve a hotel and add it to the Hotel Name
# column."

# --Retrieve Hotels from a Nearby Search
# Step 1: "Set the Parameters for a Nearby Search"
#    "We'll be using these parameters:
#        1/ API key                 (required)
#        2/ Latitude and longitude  ( "      )
#        3/ 5,000-meter radius      ( " nb max allowed is 50,000 meters;
#                                    and NOT included if rankby=distance
#                                    is specified as one of the optional
#                                    parameters)
#        4/ Type of place"          (optional*)
#    [see documentation at (nearby search requests page):
#    https://developers.google.com/places/web-service/search#PlaceSearchRequests%0D%0A]
#    (*) {see -list of supported types- from this link below (available too from w/in
#         documentation link above):
#         https://developers.google.com/places/web-service/supported_types }

# Set parameters to search for a hotel (use "lodging" as type; none for "hotel" specifically).
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# at this point
# --Practice Using a Fixed Latitude and Longitude
#   (do this in Google_Nearby_Search.ipynb) so
#   that understand where to get the hotel name

In [22]:
# ..BUT: receive `IndexError: list index out of range` error when run the code
# ..to "handle the IndexError to get the code to run ..use a try-except block"
#
# "last piece of code" to be modified as follows to "make a `try-except` block
# to handle the `IndexError`." ...(except remove leftmost set of commenting)

# # Grab the first hotel from the results and store the name.
#   try:
#     hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
#   except (IndexError):
#     print("Hotel not found... skipping.")

# NOTE: "If ...encounter more errors, add them to the `except` block--like so,
#       `(IndexError, NewError)`-- and continue running your code until there
#       are no errors."

# check the first 10 rows of the `hotel_df` DataFrame

In [23]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # # Grab the first hotel from the results and store the name.
    # hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [24]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Ciudad Bolivar,VE,80.19,8.1222,-63.5497,Hotel Edi
18,College,US,79.09,64.8569,-147.8028,Pike's Waterfront Lodge
25,Butaritari,KI,81.95,3.0707,172.7902,Isles Sunset Lodge
28,Isangel,VU,81.07,-19.5500,169.2667,Tanna Lodge
37,Avarua,CK,77.05,-21.2078,-159.7750,Paradise Inn
38,Gurupi,BR,75.74,-11.7292,-49.0686,Via Norte Hotel
39,Ixtapa,MX,86.23,20.7000,-105.2000,hotel la terminal
42,Atuona,PF,77.63,-9.8000,-139.0333,Villa Enata
44,Meulaboh,ID,75.42,4.1363,96.1285,Kost Pria
46,Tautira,PF,78.51,-17.7333,-149.1500,Pension Oaoa


In [25]:
hotel_df.tail(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
554,Laem Sing,TH,84.02,12.4816,102.0738,Laemsing Whitehouse Resort
557,Jalu,LY,83.19,29.0331,21.5482,صاحبة الفخامة
560,Bardiyah,LY,77.00,31.7561,25.0865,بورتو بردية
564,Jining,CN,81.55,35.4050,116.5814,Yunhe Star Hotel
567,San Cristobal,VE,75.38,7.7669,-72.2250,Pirineos
568,Sulangan,PH,82.98,11.1373,123.7228,Maia's Beach Resort
569,Pacific Grove,US,79.88,36.6177,-121.9166,Lovers Point Inn
570,Tiarei,PF,82.42,-17.5333,-149.3333,Le Rocher de Tahiti
571,Hay River,CA,76.98,60.8156,-115.7999,Ptarmigan Inn
574,Vieux-Habitants,GP,80.17,16.0589,-61.7659,Gîtes Dardanelle


In [26]:
hotel_df[(hotel_df["Hotel Name"]=="")].count()

City          14
Country       14
Max Temp      14
Lat           14
Lng           14
Hotel Name    14
dtype: int64

In [27]:
hotel_df.count()

City          212
Country       212
Max Temp      212
Lat           212
Lng           212
Hotel Name    212
dtype: int64

In [28]:
hotel_df[(hotel_df["Hotel Name"]!="")].count()

City          198
Country       198
Max Temp      198
Lat           198
Lng           198
Hotel Name    198
dtype: int64

In [29]:
hotel_df[(hotel_df["Hotel Name"]=="")]

,City,Country,Max Temp,Lat,Lng,Hotel Name
50,Albany,US,76.59,42.6001,-73.9662,
86,Umm Kaddadah,SD,82.67,13.6017,26.6876,
149,Poya,NC,82.20,-21.3500,165.1500,
177,Nioro,GM,80.85,13.3500,-15.7500,
330,Panguna,PG,78.91,-6.3164,155.4848,
367,Nalut,LY,88.05,30.3333,10.8500,
396,Tadine,NC,78.17,-21.5500,167.8833,
420,Massakory,TD,87.26,12.9960,15.7293,
436,Mayor Pablo Lagerenza,PY,81.45,-19.9309,-60.7718,
456,Sikasso,ML,77.49,10.9167,-7.0000,


In [30]:
# "final task will be to add pop-up markers with hotel information to a heatmap."

# Note: "For more information, see the 'Google Developers' Nearby Search
#       documentation" at the following link ...
#       https://developers.google.com/places/web-service/search#PlaceSearchRequests

# --Create a Maximum Temperature Heatmap from a Hotel DataFrame
# "Before we add pop-up markers with hotel information, let's create a heatmap using
# the maximum temperature from our `hotel_df` DataFrame by reusing the code and
# changing the DataFrame name." ...

# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
                                dissipating=False,
                                max_intensity=300,
                                point_radius=4)

fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [31]:
# "If we refer to the gmaps documentation on how to add markers (at following link:
# https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html#markers-and-symbols ),
# the syntax is `markers = gmaps.marker_layer(marker_locations)`, where the
# `marker_locations` are latitudes and longitudes. ..Let's add the markers for
# each city on top of the heatmap." ...

# modify prior cell code as follows...

# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
                                dissipating=False,
                                max_intensity=300,
                                point_radius=4)
marker_layer = gmaps.marker_layer(locations)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [32]:
# "add a pop-up marker for each city that displays the hotel name,
# city name, country, and maximum temperature. ..From the gmaps
# documentation on how to add markers (see link following...
# https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html#markers-and-symbols ),
# ...need to add an `info_box_template` that has the following syntax." ...
#      info_box_template = """
#      <dl>
#      <dt>Name</dt><dd>{column1}</dd>
#      <dt>Another name</dt><dd>{column2}</dd>
#      </dl>
#      """
#
# notes on the HTML code tags:
#   <dl> ... </dl>   is a description list
#   <dt> ... </dt>   is a term or name nested under the <dl> tag
#   <dd> ... </dd>   is used to define the <dt> term or name
#
# "For our purposes, we'll add the hotel name, city name, country code, and
# the maximum temperature values from the `hotel_df` DataFrame as the
# description definition (<dd> tag). Our code will look" as follows ...

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# "Next, add the data to the code by iterating through the `hotel_df`
# DataFrame using the `iterrows()` function. Then add the information
# to the `gmaps.marker_layer()` attribute with the locations. ..
# According to the documentation (at the following link ...
# https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html#markers-and-symbols )"
# we can modify the example code there...
#  `plant_info = [info_box_template.format(**plant) for plant in nuclear_power_plants]`
# "by replacing 'plant_info' with 'hotel_info,' and then use the `iterrows()` function
# to get the index and data in the row to add to the marker."

# Run this cell after adding the following...
# Store the DataFrame Row.
hotel_info = [info_box_template.format(
                **row) for index, row in hotel_df.iterrows()]

# to review the code above ^^^:
#  1. "We set the `hotel_info` equal to the info_box_content."
#  2. "In the list comprehension,
#     `info_box_template.format(**row) for index, row in hotel_df.iterrows()`,
#     we iterate through each 'row' of the `hotel_df` DataFrame and then
#     format the `info_box_template` (only) with the data we set to populate from
#     each row. REMEMBER (my emphasis), we are NOT (my emphasis) using every
#     row; we are only using the rows defined in the `info_box_template`,
#     which are `Hotel Name`, `City`, `Country`, and `Max Temp`.

# "Next, in the code we used to create the heatmap with markers, add
# `info_box_content=hotel_info` to the `gmaps.marker_layer()` attribute
# with the locations." (in the next cell)...


In [33]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker
# for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
                                dissipating=False,
                                max_intensity=300,
                                point_radius=4)

marker_layer = gmaps.marker_layer(locations,
                                 info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))